In [1]:
import pandas as pd
import hashlib
import nltk
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import shap


e:\GrowthAXL\venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
books=pd.read_csv('E:\\GrowthAXL\\books.csv') 


In [3]:
books.head()


,title,subtitle,author_name,author_about,publisher,publisher_date,format,page_count,language,isbn_10,...,price_numeric,description,features_text,dimensions,item_weight,images,videos,store,parent_asin,bought_together
0,Government Bullies: How Everyday Americans are...,"Paperback – August 6, 2013",Rand Paul,"Discover more of the author’s books, see simil...",Center Street,"41260th edition (August 6, 2013",Paperback,320.0,English,1455522775,...,12.99,"['Review', 'Praise for The Tea Party Goes to W...",Government regulations are out of control. The...,5.25 x 0.81 x 8 inches,9.2 ounces,[{'large': 'https://m.media-amazon.com/images/...,[],"Rand Paul (Author), Ron Paul (Foreword)",1455522775,NaN
1,Gordon Pope Thrillers: Books 1-3,"Paperback – December 17, 2020",B. B. Griffith,"B. B. Griffith was born and raised in Denver, ...",Griffith Publishing LLC,"December 17, 2020",Paperback,694.0,English,1735305855,...,24.69,"['Review', 'Selected Reviews from the Gordon P...",Gordon Pope has a special talent. | He's the o...,6 x 1.74 x 9 inches,2.21 pounds,[{'large': 'https://m.media-amazon.com/images/...,[],B. B. Griffith (Author),1735305855,NaN
2,Chart Sense: Common Sense Charts to Teach 3-8 ...,"Paperback – Illustrated, February 2, 2014",Rozlyn Linder,Follow Roz on Twitter (@rozlinder) and connect...,"Literacy Initiative, The","Illustrated edition (February 2, 2014",Paperback,182.0,English,0988950510,...,22.88,"['Review', ""This is the first book I have ever...",Chart Sense is the ultimate resource for eleme...,7.5 x 0.41 x 9.25 inches,12 ounces,[],[],Rozlyn Linder (Author),0988950510,NaN
3,Where the Sky Begins: A Novel,"Paperback – August 2, 2022",Rhys Bowen,Rhys Bowen is the New York Times bestselling a...,Lake Union Publishing,"August 2, 2022",Paperback,399.0,English,1542028841,...,6.15,"['Review', '“Bowen is a master at winning read...",A woman’s future is determined by fate and cho...,5.5 x 1 x 8.25 inches,14.1 ounces,[],[],Rhys Bowen (Author),1542028841,NaN
4,The Mystery of Garabandal: Fantasy or Fraud? G...,NaN,LR Walker,"Discover more of the author’s books, see simil...",Newtonia Publishing LLC,"First Paperback Edition (July 31, 2015",Paperback,299.0,English,0692297677,...,12.99,"['About the Author', ""LR Walker studied journa...",This 2021 update includes new interviews with ...,6 x 0.68 x 9 inches,14.3 ounces,[{'large': 'https://m.media-amazon.com/images/...,[],"LR Walker (Author), Frank Lafleur (Introduction)",0692297677,NaN


In [16]:
books.columns


Index(['book_id', 'title'], dtype='object')

In [4]:
reviews= pd.read_csv('E:\\GrowthAXL\\reviews.csv')
reviews.head()


,rating,title,text,images,asin,parent_asin,user_id,timestamp,helpful_vote,verified_purchase,date,year
0,5.0,Two of the best characters together again,I love comics. I like Batman. I like ninja tur...,[{'small_image_url': 'https://images-na.ssl-im...,1401271502,1401271502,AERNNWOCLETNLVK2GRXCJO4TO7LQ,1538608078266,0,True,2018-10-03 23:07:58.266,2018
1,5.0,Awesome book,Awesome book and I've so far had no issues wit...,[],1421520540,1421520540,AHZJ3KRU6AQ2JZE23FEJY7JAAHMQ,1673634293492,0,True,2023-01-13 18:24:53.492,2023
2,5.0,great deal,Thanks mass paperback product for it. Was expe...,[],0345428714,0345428714,AEDVXAAITTVCRDFQQH4IJTGEK4RA,1371491205000,0,True,2013-06-17 17:46:45.000,2013
3,3.0,Timely and Fun Read,Incredibly timely considering events in the Ch...,[],B00MTSC7LG,B00MTSC7LG,AE72WFCU76QJVG3N65ZF45H5ZNZA,1433807727000,2,True,2015-06-08 23:55:27.000,2015
4,5.0,Amazing story,I'm so excited I bought this book. It was a wo...,[],0441142265,0441142265,AH6PWQBQE3GQPSOKK2REUF3IZFUA,1639076612134,0,True,2021-12-09 19:03:32.134,2021


In [21]:
reviews.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 728026 entries, 0 to 728025
Data columns (total 2 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   book_id  727876 non-null  object
 1   text     727914 non-null  object
dtypes: object(2)
memory usage: 11.1+ MB


In [24]:
STUDENT_ID = "STU160"

def sha256_hex(x: str) -> str:
    return hashlib.sha256(x.encode()).hexdigest()

TARGET_HASH = sha256_hex(STUDENT_ID)[:8].upper()
print("Your 8-char hash:", TARGET_HASH)


Your 8-char hash: 5E779677


In [25]:
if "book_id" not in books.columns:
    if "parent_asin" in books.columns:
        books["book_id"] = books["parent_asin"]
    elif "asin" in books.columns:
        books["book_id"] = books["asin"]

books = books[["book_id", "title"]]

if "book_id" not in reviews.columns:
    if "parent_asin" in reviews.columns:
        reviews["book_id"] = reviews["parent_asin"]
    elif "asin" in reviews.columns:
        reviews["book_id"] = reviews["asin"]

if "text" not in reviews.columns:
    string_cols = [c for c in reviews.columns if reviews[c].dtype == "object"]
    reviews["text"] = reviews[string_cols].fillna("").agg(" ".join, axis=1)

reviews = reviews[["book_id", "text"]]
books.head(), reviews.head()


(      book_id                                              title
 0  1455522775  Government Bullies: How Everyday Americans are...
 1  1735305855                   Gordon Pope Thrillers: Books 1-3
 2  0988950510  Chart Sense: Common Sense Charts to Teach 3-8 ...
 3  1542028841                      Where the Sky Begins: A Novel
 4  0692297677  The Mystery of Garabandal: Fantasy or Fraud? G...,
       book_id                                               text
 0  1401271502  I love comics. I like Batman. I like ninja tur...
 1  1421520540  Awesome book and I've so far had no issues wit...
 2  0345428714  Thanks mass paperback product for it. Was expe...
 3  B00MTSC7LG  Incredibly timely considering events in the Ch...
 4  0441142265  I'm so excited I bought this book. It was a wo...)

In [27]:
reviews[reviews["text"].str.contains("5E779677", case=False, na=False)]


,book_id,text


In [26]:
mask = reviews["text"].astype(str).str.contains(TARGET_HASH, case=False, na=False)
fake_review = reviews[mask]

if fake_review.empty:
    print("ERROR: Fake review not found.")
else:
    display(fake_review)

book_id = fake_review.iloc[0]["book_id"]
title = books[books["book_id"] == book_id].iloc[0]["title"]

clean_title = "".join(title.split())[:8]
FLAG1 = sha256_hex(clean_title)
FLAG2 = f"FLAG2{{{TARGET_HASH}}}"

FLAG1, FLAG2


ERROR: Fake review not found.


IndexError: single positional indexer is out-of-bounds

In [ ]:
book_reviews = reviews[reviews["book_id"] == book_id].copy()

def label_suspicious(text: str) -> int:
    t = str(text).lower()
    words = t.split()
    short = len(words) < 12
    superlatives = any(w in t for w in ["best", "amazing", "awesome", "incredible", "perfect"])
    return 1 if (short and superlatives) else 0

book_reviews["is_suspicious"] = book_reviews["text"].apply(label_suspicious)

vectorizer = TfidfVectorizer(stop_words="english")
X = vectorizer.fit_transform(book_reviews["text"])
y = book_reviews["is_suspicious"].values

model = LogisticRegression(max_iter=1000)
model.fit(X, y)

explainer = shap.LinearExplainer(model, X, feature_perturbation="interventional")
shap_values = explainer.shap_values(X)
if isinstance(shap_values, list):
    shap_values = shap_values[0]

genuine_idx = book_reviews[book_reviews["is_suspicious"] == 0].index
if len(genuine_idx) == 0:
    genuine_idx = book_reviews.index

mean_shap = shap_values[genuine_idx].mean(axis=0)
top_idx = mean_shap.argsort()[:3]
feature_names = vectorizer.get_feature_names_out()
top_words = [feature_names[i] for i in top_idx]

student_num = STUDENT_ID.replace("STU", "")
FLAG3_raw = "".join(top_words) + student_num
FLAG3 = "FLAG3{" + sha256_hex(FLAG3_raw)[:10] + "}"

top_words, FLAG3


In [ ]:
with open("flags.txt", "w") as f:
    f.write(f"FLAG1 = {FLAG1}\n")
    f.write(f"FLAG2 = {FLAG2}\n")
    f.write(f"FLAG3 = {FLAG3}\n")

print("flags.txt saved.")
